In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque
from sklearn.neighbors import NearestNeighbors

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
df1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2018-2019.csv')
df1.Date = pd.to_datetime(df1.Date)

df2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2019-2020.csv')
df2.Date = pd.to_datetime(df2.Date)

df = df1.append(df2)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2018-07-30 07:00:00   1.31106   1.31109  1.31061    1.31069   1.31131   
1 2018-07-30 07:05:00   1.31075   1.31089  1.30964    1.31032   1.31093   
2 2018-07-30 07:10:00   1.31038   1.31102  1.31038    1.31094   1.31054   
3 2018-07-30 07:15:00   1.31094   1.31136  1.31091    1.31098   1.31112   
4 2018-07-30 07:20:00   1.31103   1.31150  1.31103    1.31134   1.31121   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.31138  1.31077    1.31086   1.31118   1.31124  1.31070    1.31078  
1   1.31107  1.30981    1.31047   1.31084   1.31098  1.30972    1.31040  
2   1.31119  1.31054    1.31111   1.31046   1.31110  1.31046    1.31102  
3   1.31155  1.31107    1.31114   1.31103   1.31145  1.31099    1.31106  
4   1.31169  1.31121    1.31152   1.31112   1.31160  1.31112    1.31143  
---------------------------------------------------------------------------
                      Date  B

In [5]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [6]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())

0.0    111783
1.0     18586
2.0     17853
Name: fractal, dtype: int64


In [13]:
value_per_pip = 1.0
max_pips_to_risk = 0.0050
amounts_per_day = [-4, -5, -6]

In [14]:
class AatTuple:
    def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
        self.time = time
        self.pips_risked = pips_risked
        self.pips_gained = pips_gained
        self.total_profit = total_profit
        self.adx_assumption = adx_assumption
        self.ema_assumption = ema_assumption
        self.rsi_assumption = rsi_assumption

In [15]:
# Use trade type in KNN?
# Normalize the data in KNN?

In [16]:
training_data = []

In [17]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def get_end_date(j):
  return df.loc[df.index[j + 1], 'Date'] if j + 1 < len(df) else None

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_training_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, lam=0.5):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None
    current_training_points = []
    step = 0

    i = 2

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [macd2, macdsignal2, macd1, macdsignal1]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_adx, curr_ml, curr_mh, curr_ema, curr_rsi  = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close', 'adx', 'Mid_Low', 'Mid_High', 'ema200', 'rsi']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 1:
                        curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                        curr_fractal_distance = curr_ao - curr_pullback

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_ask_open)
                    stop_loss = round(pullback - pullback_cushion, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

                            current_training_points = []
                            step = 0

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_sells = True

              if all_sells:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 2:
                        curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                        curr_fractal_distance = curr_pullback - curr_bo

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_bid_open)
                    stop_loss = round(pullback + pullback_cushion, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

                            current_training_points = []
                            step = 0

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          d_fee = calculate_day_fees(trade['start_date'], curr_date)
          day_fees += d_fee

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

          for point in current_training_points:
              point.total_profit = trade_amount + d_fee

          training_data.extend(current_training_points)

        # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
        #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          d_fee = calculate_day_fees(trade['start_date'], curr_date)
          day_fees += d_fee

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

          for point in current_training_points:
              point.total_profit = trade_amount + d_fee

          training_data.extend(current_training_points)

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          d_fee = calculate_day_fees(trade['start_date'], curr_date)
          day_fees += d_fee

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

          for point in current_training_points:
              point.total_profit = trade_amount + d_fee

          training_data.extend(current_training_points)

        # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
        #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          d_fee = calculate_day_fees(trade['start_date'], curr_date)
          day_fees += d_fee

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

          for point in current_training_points:
              point.total_profit = trade_amount + d_fee

          training_data.extend(current_training_points)

        if trade is not None:
          pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

          z_adx_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].adx_assumption * lam
          z_ema_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].ema_assumption * lam
          z_rsi_prev = 0 if len(current_training_points) == 0 else current_training_points[-1].rsi_assumption * lam

          adx_holds = int(curr_adx > adx_cutoff)
          ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
          rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

          adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
          ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
          rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

          current_training_points.append(AatTuple(step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption))

          step += 1

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [18]:
# ----------------------------------------------------------------------------------------------------
# Run training simulation
# ----------------------------------------------------------------------------------------------------
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_training_simulation(2, 5, 15, 5, 20, 2)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))

-6510.000000000019
Num buys: 223
Num sells: 249
Num trades: 472
Num wins: 141
Num losses: 331
Win streak: 6
Loss streak: 12


In [19]:
len(training_data)

62285

In [20]:
# Repeate, but with test data
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [21]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())

0.0    55662
1.0     9387
2.0     9014
Name: fractal, dtype: int64


In [24]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def get_end_date(j):
  return df.loc[df.index[j + 1], 'Date'] if j + 1 < len(df) else None

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_regular_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 2

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [macd2, macdsignal2, macd1, macdsignal1]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 1:
                        curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                        curr_fractal_distance = curr_ao - curr_pullback

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_ask_open)
                    stop_loss = round(pullback - pullback_cushion, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_sells = True

              if all_sells:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 2:
                        curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                        curr_fractal_distance = curr_pullback - curr_bo

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_bid_open)
                    stop_loss = round(pullback + pullback_cushion, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
        #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
        #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [25]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_regular_simulation(2, 5, 15, 5, 20, 2)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))

4641.000000000007
Num buys: 104
Num sells: 120
Num trades: 224
Num wins: 84
Num losses: 140
Win streak: 5
Loss streak: 9
Average pips risked: 0.002671205357142856


In [51]:
# class AatTuple:
#     def __init__(self, time, pips_risked, pips_gained, total_profit, adx_assumption, ema_assumption, rsi_assumption):
#         self.time = time
#         self.pips_risked = pips_risked
#         self.pips_gained = pips_gained
#         self.total_profit = total_profit
#         self.adx_assumption = adx_assumption
#         self.ema_assumption = ema_assumption
#         self.rsi_assumption = rsi_assumption

# x_train = []
# for tup in training_data:
#     curr_x = [tup.time, tup.pips_risked, tup.pips_gained, tup.total_profit, tup.adx_assumption, tup.ema_assumption, tup.rsi_assumption]
#     x_train.append(curr_x)

# x_train = np.array(x_train)


# def distance_func(x, y):
#     time_dist = 1000 * abs(x[0] - y[0])
#     curr_pips_dist = 2 * abs(x[2] - y[2])
#     pips_risked_dist = 3 * abs(x[1] - y[1])
#     adx_dist = abs(x[4] - y[4])
#     ema_dist = abs(x[5] - y[5])
#     rsi_dist = abs(x[6] - y[6])

#     return sum([time_dist, curr_pips_dist, pips_risked_dist, adx_dist, ema_dist, rsi_dist])

# knn = NearestNeighbors(n_neighbors=15, metric=distance_func)
# knn.fit(x_train)

x_train = []
y_train = []
for tup in training_data:
    curr_x = [tup.time, tup.pips_risked, tup.pips_gained, tup.adx_assumption, tup.ema_assumption, tup.rsi_assumption]
    x_train.append(curr_x)
    curr_y = tup.total_profit
    y_train.append(curr_y)

x_train = np.array(x_train)
y_train = np.array(y_train)

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_new = scaler.fit_transform(x_train)

mlp = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500)
mlp.fit(x_new, y_train)

/Users/mymac/Forex_Robot/simulations/venv/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(max_iter=500)

In [52]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def get_end_date(j):
  return df.loc[df.index[j + 1], 'Date'] if j + 1 < len(df) else None

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_test_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, lam=0.5):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None
    estimated_gains = []
    step = 0
    current_tuples = []

    i = 2

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [macd2, macdsignal2, macd1, macdsignal1]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_adx, curr_ml, curr_mh, curr_ema, curr_rsi  = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close', 'adx', 'Mid_Low', 'Mid_High', 'ema200', 'rsi']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 1:
                        curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                        curr_fractal_distance = curr_ao - curr_pullback

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_ask_open)
                    stop_loss = round(pullback - pullback_cushion, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

                            step = 0
                            current_tuples = []

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_sells = True

              if all_sells:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 2:
                        curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                        curr_fractal_distance = curr_pullback - curr_bo

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_bid_open)
                    stop_loss = round(pullback + pullback_cushion, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

                            step = 0
                            current_tuples = []

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
        #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
        #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None:
          pips_gained_so_far = (curr_bid_close - trade['open_price']) * trade['n_units'] if trade['trade_type'] == 'buy' else (trade['open_price'] - curr_ask_close) * trade['n_units']

          z_adx_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].adx_assumption * lam
          z_ema_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].ema_assumption * lam
          z_rsi_prev = 0 if len(current_tuples) == 0 else current_tuples[-1].rsi_assumption * lam

          adx_holds = int(curr_adx > adx_cutoff)
          ema_holds = int(curr_ml > curr_ema if trade['trade_type'] == 'buy' else curr_mh < curr_ema)
          rsi_holds = int(curr_rsi < 70 if trade['trade_type'] == 'buy' else curr_rsi > 30)

          adx_assumption = z_adx_prev + ((1 - lam) * adx_holds)
          ema_assumption = z_ema_prev + ((1 - lam) * ema_holds)
          rsi_assumption = z_rsi_prev + ((1 - lam) * rsi_holds)

          # current_tuples.append(AatTuple(step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption))
          # current_tuple = np.array([step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption])
          # distances, neighbor_indices = knn.kneighbors(current_tuple.reshape(1, -1), 15)
          # estimated_gain = 0
          # distance_sum = 0

          # for p in range(len(neighbor_indices[0])):
          #   l = neighbor_indices[0][p]
          #   l_dist = distances[0][p]
          #   estimated_gain += (x_train[l][3] / l_dist)
          #   distance_sum += 1 / l_dist

          # estimated_gain /= distance_sum
          current_tuples.append(AatTuple(step, trade['pips_risked'], pips_gained_so_far, 0, adx_assumption, ema_assumption, rsi_assumption))
          current_tuple = np.array([step, trade['pips_risked'], pips_gained_so_far, adx_assumption, ema_assumption, rsi_assumption])
          estimated_gain = mlp.predict(scaler.transform(current_tuple.reshape(1, -1)))[0]
          estimated_gains.append(estimated_gain)

          day_fees_now = calculate_day_fees(trade['start_date'], curr_date)

          if estimated_gain < pips_gained_so_far + day_fees_now:
            trade_amount = pips_gained_so_far * value_per_pip
            reward += trade_amount
            day_fees += day_fees_now

            n_wins += 1 if trade_amount >= 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount >= 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

          step += 1

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, estimated_gains

In [53]:
reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, estimated_gains = run_test_simulation(2, 5, 15, 5, 20, 2)

print(reward)
print('Num buys: ' + str(n_sells))
print('Num sells: ' + str(n_buys))
print('Num trades: ' + str(n_buys + n_sells))
print('Num wins: ' + str(n_wins))
print('Num losses: ' + str(n_losses))
print('Win streak: ' + str(win_streak))
print('Loss streak: ' + str(loss_streak))
print('Average pips risked: ' + str(sum(pips_risked) / len(pips_risked)))

-4358.000000000073
Num buys: 216
Num sells: 264
Num trades: 480
Num wins: 140
Num losses: 340
Win streak: 4
Loss streak: 14
Average pips risked: 0.002612104166666662


In [54]:
sum(estimated_gains) / len(estimated_gains)

61.364180371255415

In [25]:
min(estimated_gains)

-218.74063109106186

In [26]:
max(estimated_gains)

585.7045995724723